## Installing required libraries

In [ ]:
!pip install autogen-agentchat
!pip install autogen-ext[ollama]

In [ ]:
!pip install ollama

## Check ollama

In [ ]:
import ollama

result = ollama.generate(model='qwen3:14b', prompt ='What is your name? respond in 1 line only')
print(result['response'])

## Create agents

In [ ]:
from autogen_ext.models.ollama import OllamaChatCompletionClient

#Create a client with local Qwen

ollama_client = OllamaChatCompletionClient(
    model="qwen3:14b",
    api_key="placeholder",
    base_url="http://localhost:11434/v1",
    model_info={
        "function_calling": True,
        "json_output": True,
        "vision": False,
        "family": "unknown"
    }
)

In [ ]:
# Get ARR data
import numpy as np
import pandas as pd

def get_ARR_data():
    """Get the ARR data in a pandas dataframe format"""
    ARR_data = pd.read_csv("Customer_Level_ARR_Changes.csv")
    return ARR_data

In [ ]:
# Checking ARR function

arr_dataframe = get_ARR_data()
arr_dataframe.head()

In [ ]:
# define a function to calculate retention

def analyze_arr_rollforward():
    
    """
    Analyzes ARR retention and returns metrics per year:
    - Total ARR
    - Top 2 customer losses (with amount)
    - Top 2 new wins (with amount)
    - Gross Retention %
    - Net Retention %
        
    Returns:
        dict: A dictionary of metrics per year
    """
    
    
    results = {}
    
    df=get_ARR_data()
    year_columns = ["Year 1 ARR ($)", "Year 2 ARR ($)", "Year 3 ARR ($)"]
    customer_id_col="Customer ID"
    
    for i in range(1, len(year_columns)):
        prev_year = year_columns[i - 1]
        curr_year = year_columns[i]

        # Merge current and previous year ARR for all customers
        prev_df = df[[customer_id_col, prev_year]].copy()
        curr_df = df[[customer_id_col, curr_year]].copy()
        merged = pd.merge(prev_df, curr_df, on=customer_id_col, how='outer').fillna(0)

        # Compute delta
        merged['delta'] = merged[curr_year] - merged[prev_year]

        # Identify categories
        retained = (merged[prev_year] > 0) & (merged[curr_year] > 0)
        churned = (merged[prev_year] > 0) & (merged[curr_year] == 0)
        new_customers = (merged[prev_year] == 0) & (merged[curr_year] > 0)

        starting_arr = merged.loc[retained | churned, prev_year].sum()
        upsell = merged.loc[retained & (merged['delta'] > 0), 'delta'].sum()
        downsell = merged.loc[retained & (merged['delta'] < 0), 'delta'].sum()
        churn = merged.loc[churned, prev_year].sum() * -1

        # Metrics
        total_arr = merged[curr_year].sum()
        gross_retention = ((starting_arr + downsell + churn) / starting_arr * 100) if starting_arr else 0
        net_retention = ((starting_arr + downsell + churn + upsell) / starting_arr * 100) if starting_arr else 0

        # Top 2 losses
        merged['loss_amount'] = merged[prev_year] - merged[curr_year]
        top_losses = merged.sort_values('loss_amount', ascending=False).head(2)[[customer_id_col, 'loss_amount']].to_dict(orient='records')

        # Top 2 new wins
        wins = merged[new_customers].copy()
        wins['win_amount'] = wins[curr_year]
        top_wins = wins.sort_values('win_amount', ascending=False).head(2)[[customer_id_col, 'win_amount']].to_dict(orient='records')

        # Store in results
        results[f"{prev_year} to {curr_year}"] = {
            "Total ARR": round(total_arr, 2),
            "Starting ARR": round(starting_arr, 2),
            #"Upsell": round(upsell, 2),
            #"Downsell": round(downsell, 2),
            #"Churn": round(churn,2),
            "Top 2 Customer Lost or downsell": top_losses,
            "Top 2 New Wins": top_wins,
            "Gross Retention %": round(gross_retention, 2),
            "Net Retention %": round(net_retention, 2)
        }

    return results



In [ ]:
#check analyze ARR function
retention_metrics = analyze_arr_rollforward()
print(retention_metrics)

In [ ]:
from autogen_agentchat.agents import CodeExecutorAgent, AssistantAgent
#from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
#from autogen_core import CancellationToken
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console

In [ ]:
#Creating a planner agent

planner = AssistantAgent(
    "planner",
    model_client=ollama_client,
    handoffs=["financial_analyst", "data_analyst", "writer"],
    system_message="""You are a ARR and retention planning coordinator.
    Coordinate ARR and retention analysis by delegating to specialized agents:
    - financial_analyst: For ARR and retention analysis and generating insights
    - data_analyst: For calculating ARR and retention metrics
    - writer: For writing the final report in a concise manner
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Provide the user with the final report and Respond with TERMINATE when analysis is complete.""",
)


#Creating the data_analyst

data_analyst = AssistantAgent(
    "data_analyst",
    model_client=ollama_client,
    handoffs=['planner'],
    tools=[analyze_arr_rollforward],
    system_message="""You are a data analyst.
    You will generate ARR and retention metrics using the tool analyze_ARR_rollfoward.
    Always handoff back to the planner when the analysis is complete.
    """,
)

# Creating financial analyst agent

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=ollama_client,
    handoffs=["planner"],
    tools=[get_ARR_data],
    system_message="""You are a financial analyst.
    Analyze the ARR data using the get_ARR_data.
    Analyze the ARR and retention metrics calculated by the data_analyst and provided by planner
    Provide insights on ARR and retention metrics.
    Always handoff back to the planner when analysis in complete.
    """,
)

#Creating writer agent
writer = AssistantAgent(
    "writer",
    model_client=ollama_client,
    handoffs=["planner"],
    system_message="""You are a due diligence report writer.
    Compile the ARR analysis and findings into clear, concise report.
    Always handoff back to the planner when writing is complete.""",
)



In [ ]:
# Define termination condition

text_termination = TextMentionTermination("Terminate")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, data_analyst, writer], termination_condition=termination    
)

In [ ]:
task = "Conduct ARR analysis based on the data provided for software company."
await Console(research_team.run_stream(task=task))
await ollama_client.close()